In [1]:
import json
import numpy as np
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
import time


from src.experiments.aux_code import *
from datasets.get_datasets import *
from src.brkga.genetic import genetic as brkga
from src.brkga_variation.genetic import genetic as brkga_var
from src.s_genetic.genetic import *
from src.predicate import *

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


In [2]:
n_rounds = 5
EXPERIMENT_NAME = 'imdb_uwcse'

In [3]:
def get_k_best_individuals(population, k):
    best_ind = []
    all_fitness = []
    for i in range(0, len(population)):
        all_fitness.append((population[i].fitness.values[0], i))
    
    sorted_ind = sorted(all_fitness, reverse=False, key=lambda tup: tup[0])
    for i in range(0, k):
        best_ind.append(population[sorted_ind[i][1]])
    return best_ind

In [4]:
source='workedunder'
target='advisedby'

bk = json.loads(open('src/experiments/kb.txt').readline())
kb_source = bk['imdb']
kb_target = bk['uwcse']

pred_target = create_pred_target(kb_target)

In [5]:
imdb_dataset = datasets.load('imdb', kb_source, target=source, seed=441773, balanced=0) #facts, pos, neg
uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773, balanced=0) #facts, pos, neg


ss = []
with open('src/experiments/structures.json', 'r') as f:
    ss = json.loads(f.readline())
    
src_struct = copy.deepcopy(ss[EXPERIMENT_NAME])
new_src_struct = []
for i in range(0, len(src_struct)):
    new_src_struct.append(define_individual(src_struct[i], i))  
structured_src = src_struct



In [6]:
train_pos, train_neg, train_facts = uwcse_dataset[1], uwcse_dataset[2], uwcse_dataset[0]
train_pos, train_neg, train_facts = get_train_test(train_pos, train_neg, train_facts,  n_folds=3)

save_examples(train_pos, train_neg, f'groot_experiments/{EXPERIMENT_NAME}_{source}_{target}/')

# Testing s_genetic

In [7]:
mutation_rate_list = [round(x, 2) for x in list(np.arange(0.1, 0.4, 0.05))]
crossover_rate_list = [round(x, 2) for x in list(np.arange(0.6, 0.95, 0.05)) + [0.95]]

space  = [Categorical([10, 30, 50], name='num_individuals'),
          Categorical(mutation_rate_list, name='mutation_rate'),
          Categorical(crossover_rate_list, name='crossover_rate')]

In [9]:
for fold in range(len(train_pos)):
    val_pos, val_neg, val_facts = get_train_test([train_pos[fold]], [train_neg[fold]], [train_facts[fold]],  n_folds=3)
    
    @use_named_args(space)
    def objective(**params):
        res = genetic(new_src_struct, target, source, val_pos, val_neg, 
                          val_facts, kb_source, kb_target, pred_target,
                          NUM_GEN=14, pop_size=params['num_individuals'], crossover=params['crossover_rate'],
                          mutation=params['mutation_rate'], crossover_type='tree_ind', revision='guided')

        return res[1][-1]
    res_gp = gp_minimize(objective, space, n_calls=10, random_state=0)

    print("Best score=%.4f " % res_gp.fun)
    print(f"FOLD {fold} - BEST RESULT: {res_gp.x}")
    
    test = []
    ttrain = []
    test_pos = []
    test_neg = [] 
    test_facts = []
    for index in range(0, len(train_pos)):
        if index == fold:
            ttrain = [train_pos[index], train_neg[index], train_facts[index]]
        else:
            test_pos.extend(train_pos[index])
            test_neg.extend(train_neg[index])
            test_facts.extend(train_facts[index])
    test = [test_pos, test_neg, test_facts]
    
    train_pos_gen = [ttrain[0], test[0]]
    train_neg_gen = [ttrain[1], test[1]]
    train_facts_gen = test[2]
    
    for _round in range(0, n_rounds):
        print(f"ROUND {str(_round+1)}")


        res_s_genetic = genetic(new_src_struct, target, source, 
                        train_pos_gen, train_neg_gen, train_facts_gen, 
                        kb_source, kb_target, pred_target,
                        NUM_GEN=14, pop_size=res_gp.x[0], 
                        mutation=res_gp.x[1], crossover=res_gp.x[2],
                        crossover_type='tree_ind', revision='guided')
        

        final_results = {}
        final_results[f'{source}->{target}'] = res_s_genetic

        individuals = get_k_best_individuals(res_s_genetic[0].population, 3)
        
        n_ind = 1
        for individual in individuals:
            rrefine = []
            rtransfer = []
            print("INDIVIDUO ", n_ind)
            refine, transfer = get_refine_transfer(individual)
            rrefine.append(refine)
            rtransfer.append(transfer)
            res = []
            inf = []
            
            res_ =  test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
            res.append(res_)

            thisFile = f'boostsrl/test/results_{target}.db'
            base = os.path.splitext(thisFile)[0]
            os.rename(thisFile, base + ".txt")
            tt = open(f'boostsrl/test/results_{target}.txt', 'r').readlines()
            final = []
            for i in tt:
                final.append(i.replace('\n', ''))
            inf.append(final)


            final_results[f'test:{source}->{target}'] = res
            final_results[f'refine:{source}->{target}'] = rrefine
            final_results[f'transfer:{source}->{target}'] = rtransfer
            final_results[f'inf:{source}->{target}'] = inf
            save_groot_results(f'groot_experiments/{EXPERIMENT_NAME}_{source}_{target}/s_genetic/train_fold_{fold}/{str(res_gp.x[2])}_{str(res_gp.x[1])}_{str(res_gp.x[0])}_15', n_ind, final_results, source, target)
            n_ind += 1

/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


GENERATION:  0
MELHOR RESULTADO:  -0.381821
BEST: {'m_auc_pr': 0.381821, 'm_auc_roc': 0.926487, 'm_cll': -0.347412, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.2027275, 's_f1': 0.0}
GENERATION:  1
MELHOR RESULTADO:  -0.381821
BEST: {'m_auc_pr': 0.381821, 'm_auc_roc': 0.926487, 'm_cll': -0.34766, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.2027275, 's_f1': 0.0}
GENERATION:  2
MELHOR RESULTADO:  -0.381821
BEST: {'m_auc_pr': 0.381821, 'm_auc_roc': 0.926487, 'm_cll': -0.34766, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.2027275, 's_f1': 0.0}
GENERATION:  3
MELHOR RESULTADO:  -0.382841
BEST: {'m_auc_pr': 0.382841, 'm_auc_roc': 0.926889, 'm_cll': -0.256995, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 

BEST: {'m_auc_pr': 0.381821, 'm_auc_roc': 0.926487, 'm_cll': -0.3429, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.2027275, 's_f1': 0.0}
GENERATION:  2
MELHOR RESULTADO:  -0.381821
BEST: {'m_auc_pr': 0.381821, 'm_auc_roc': 0.926487, 'm_cll': -0.3429, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.2027275, 's_f1': 0.0}
GENERATION:  3
MELHOR RESULTADO:  -0.381821
BEST: {'m_auc_pr': 0.381821, 'm_auc_roc': 0.926487, 'm_cll': -0.354707, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.2027275, 's_f1': 0.0}
GENERATION:  4
MELHOR RESULTADO:  -0.381821
BEST: {'m_auc_pr': 0.381821, 'm_auc_roc': 0.926487, 'm_cll': -0.354707, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.202727

BEST: {'m_auc_pr': 0.388218, 'm_auc_roc': 0.931104, 'm_cll': -0.336148, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.2027275, 's_f1': 0.0}
GENERATION:  10
MELHOR RESULTADO:  -0.388218
BEST: {'m_auc_pr': 0.388218, 'm_auc_roc': 0.931104, 'm_cll': -0.336148, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.2027275, 's_f1': 0.0}
GENERATION:  11
MELHOR RESULTADO:  -0.388218
GENERATION:  0
MELHOR RESULTADO:  -0.381821
BEST: {'m_auc_pr': 0.381821, 'm_auc_roc': 0.926487, 'm_cll': -0.334206, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.2027275, 's_f1': 0.0}
GENERATION:  1
MELHOR RESULTADO:  -0.381821
BEST: {'m_auc_pr': 0.381821, 'm_auc_roc': 0.926487, 'm_cll': -0.334206, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0

BEST: {'m_auc_pr': 0.417456, 'm_auc_roc': 0.952664, 'm_cll': -0.260469, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.2027275, 's_f1': 0.0}
GENERATION:  10
MELHOR RESULTADO:  -0.417456
BEST: {'m_auc_pr': 0.417456, 'm_auc_roc': 0.952664, 'm_cll': -0.260469, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.2027275, 's_f1': 0.0}
GENERATION:  11
MELHOR RESULTADO:  -0.417456
BEST: {'m_auc_pr': 0.417456, 'm_auc_roc': 0.952664, 'm_cll': -0.260469, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.2027275, 's_f1': 0.0}
GENERATION:  12
MELHOR RESULTADO:  -0.417456
BEST: {'m_auc_pr': 0.417456, 'm_auc_roc': 0.952664, 'm_cll': -0.359761, 'm_rec': 1.0, 'm_pred': 0.2972725, 'm_f1': 0.172757, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0

...

BEST: {'m_auc_pr': 0.147807, 'm_auc_roc': 0.907539, 'm_cll': -0.269589, 'm_rec': 0.807692, 'm_pred': 0.3255395, 'm_f1': 0.254545, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.17446050000000002, 's_f1': 0.0}
GENERATION:  3
MELHOR RESULTADO:  -0.147807
BEST: {'m_auc_pr': 0.147807, 'm_auc_roc': 0.907539, 'm_cll': -0.269589, 'm_rec': 0.807692, 'm_pred': 0.3255395, 'm_f1': 0.254545, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.17446050000000002, 's_f1': 0.0}
GENERATION:  4
MELHOR RESULTADO:  -0.147807
BEST: {'m_auc_pr': 0.147807, 'm_auc_roc': 0.907539, 'm_cll': -0.269589, 'm_rec': 0.807692, 'm_pred': 0.3255395, 'm_f1': 0.254545, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.17446050000000002, 's_f1': 0.0}
GENERATION:  5
MELHOR RESULTADO:  -0.147807
BEST: {'m_auc_pr': 0.147807, 'm_auc_roc': 0.907539, 'm_cll': -0.270786, 'm_rec': 0.807692, 'm_pred': 0.3255395, 'm_f1': 0.254545, 's_auc_pr': 0.0, 's_auc_roc

/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


GENERATION:  0
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.313403, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  1
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.314831, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  2
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.316931, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  3
MELHOR RESULTADO:  -0.206035
BEST: {'m_auc_pr': 0.206035, 'm_auc_roc': 0.906071, 'm_cll': -0.264565, 'm_rec': 1.0, 'm_pred': 0.2973815, 'm_f1': 0.173121, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_c

BEST: {'m_auc_pr': 0.229211, 'm_auc_roc': 0.927687, 'm_cll': -0.283021, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  2
MELHOR RESULTADO:  -0.229211
BEST: {'m_auc_pr': 0.229211, 'm_auc_roc': 0.927687, 'm_cll': -0.283021, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  3
MELHOR RESULTADO:  -0.34408
BEST: {'m_auc_pr': 0.34408, 'm_auc_roc': 0.937026, 'm_cll': -0.257255, 'm_rec': 0.842105, 'm_pred': 0.3249415, 'm_f1': 0.254473, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.1750585, 's_f1': 0.0}
GENERATION:  4
MELHOR RESULTADO:  -0.34408
BEST: {'m_auc_pr': 0.34408, 'm_auc_roc': 0.937026, 'm_cll': -0.257255, 'm_rec': 0.842105, 'm_pred': 0.3249415, 'm_f1': 0.254473, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.

BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.323601, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  3
MELHOR RESULTADO:  -0.219413
BEST: {'m_auc_pr': 0.219413, 'm_auc_roc': 0.922668, 'm_cll': -0.306689, 'm_rec': 1.0, 'm_pred': 0.307927, 'm_f1': 0.20765, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.192073, 's_f1': 0.0}
GENERATION:  4
MELHOR RESULTADO:  -0.219413
BEST: {'m_auc_pr': 0.219413, 'm_auc_roc': 0.922668, 'm_cll': -0.308304, 'm_rec': 1.0, 'm_pred': 0.307927, 'm_f1': 0.20765, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.192073, 's_f1': 0.0}
GENERATION:  5
MELHOR RESULTADO:  -0.219413
BEST: {'m_auc_pr': 0.219413, 'm_auc_roc': 0.922668, 'm_cll': -0.308304, 'm_rec': 1.0, 'm_pred': 0.307927, 'm_f1': 0.20765, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.192073, 's_f

# BRKGA

In [10]:
num_elite_list = [np.around(x, 2) for x in list(np.arange(0.1, 0.25, 0.05)) + [0.25]]
num_mutation_list = [np.around(x, 2) for x in list(np.arange(0.1, 0.3, 0.05)) + [0.3]]
mutation_rate_list = [round(x, 2) for x in list(np.arange(0.1, 0.4, 0.05))]
crossover_rate_list = [np.around(x, 2) for x in list(np.arange(0.6, 0.95, 0.05)) + [0.95]]

space  = [Categorical([10, 30, 50], name='num_individuals'),
          Categorical(mutation_rate_list, name='mutation_rate'),
          Categorical(crossover_rate_list, name='crossover_rate'),
          Categorical(num_elite_list, name='num_elite'),
          Categorical(num_mutation_list, name='num_mutation')]

In [11]:
for fold in range(len(train_pos)):
    val_pos, val_neg, val_facts = get_train_test([train_pos[fold]], [train_neg[fold]], [train_facts[fold]],  n_folds=3)
    
    @use_named_args(space)
    def objective(**params):
        res = brkga(new_src_struct, target, source, train_pos, train_neg, 
                          train_facts, kb_source, kb_target, pred_target,
                          NUM_GEN=14, pop_size=params['num_individuals'], crossover=params['crossover_rate'],
                          mutation=params['mutation_rate'], num_elite=params['num_elite'], 
                          num_mutation=params['num_mutation'])

        return res[1][-1]
    res_gp = gp_minimize(objective, space, n_calls=10, random_state=0)

    print("Best score=%.4f " % res_gp.fun)
    print(f"FOLD {fold} - BEST RESULT: {res_gp.x}")
    
    test = []
    ttrain = []
    test_pos = []
    test_neg = [] 
    test_facts = []
    for index in range(0, len(train_pos)):
        if index == fold:
            ttrain = [train_pos[index], train_neg[index], train_facts[index]]
        else:
            test_pos.extend(train_pos[index])
            test_neg.extend(train_neg[index])
            test_facts.extend(train_facts[index])
    test = [test_pos, test_neg, test_facts]
    
    train_pos_gen = [ttrain[0], test[0]]
    train_neg_gen = [ttrain[1], test[1]]
    train_facts_gen = test[2]
    
    for _round in range(0, n_rounds):
        print(f"ROUND {str(_round+1)}")


        res_brkga = brkga(new_src_struct, target, source, 
                    train_pos, train_neg, train_facts, 
                    kb_source, kb_target, pred_target,
                    NUM_GEN=14, pop_size=res_gp.x[0], 
                    mutation=res_gp.x[1], crossover=res_gp.x[2],
                    num_elite=res_gp.x[3], 
                    num_mutation=res_gp.x[4])

        final_results = {}
        final_results[f'{source}->{target}'] = res_brkga

        individuals = get_k_best_individuals(res_brkga[0].population, 3)
        
        n_ind = 1
        for individual in individuals:
            rrefine = []
            rtransfer = []
            print("INDIVIDUO ", n_ind)
            refine, transfer = get_refine_transfer(individual)
            rrefine.append(refine)
            rtransfer.append(transfer)
            res = []
            inf = []
            
            res_ =  test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
            res.append(res_)

            thisFile = f'boostsrl/test/results_{target}.db'
            base = os.path.splitext(thisFile)[0]
            os.rename(thisFile, base + ".txt")
            tt = open(f'boostsrl/test/results_{target}.txt', 'r').readlines()
            final = []
            for i in tt:
                final.append(i.replace('\n', ''))
            inf.append(final)


            final_results[f'test:{source}->{target}'] = res
            final_results[f'refine:{source}->{target}'] = rrefine
            final_results[f'transfer:{source}->{target}'] = rtransfer
            final_results[f'inf:{source}->{target}'] = inf
            save_groot_results(f'groot_experiments/{EXPERIMENT_NAME}_{source}_{target}/brkga/train_fold_{fold}/{str(res_gp.x[2])}_{str(res_gp.x[1])}_{str(res_gp.x[0])}_15_{str(res_gp.x[3])}_{str(res_gp.x[4])}', n_ind, final_results, source, target)
            n_ind += 1

/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


GENERATION:  0
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.286754, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  1
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.303451, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  2
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.304977, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  3
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.306511, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cl

BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.311474, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  8
MELHOR RESULTADO:  -0.205038
GENERATION:  0
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.306329, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  1
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.306754, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  2
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.305027, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cl

BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.271267, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  7
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.273403, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  8
MELHOR RESULTADO:  -0.205038
GENERATION:  0
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.303586, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  1
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.289769, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cl

/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


...

BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.315583, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  6
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.308666, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  7
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.311684, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  8
MELHOR RESULTADO:  -0.205038
GENERATION:  0
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.308716, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cl

BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.273478, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  5
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.285574, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  6
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.272945, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  7
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.281452, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, '

/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


GENERATION:  0
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.31155, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  1
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.308863, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  2
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.304671, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  3
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.303106, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll

BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.305099, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  7
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.308779, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  8
MELHOR RESULTADO:  -0.205038
INDIVIDUO  1
INDIVIDUO  2
INDIVIDUO  3
ROUND 5
GENERATION:  0
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.306427, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  1
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.303685, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.1

# BRKGA VARIATION

In [12]:
num_elite_list = [np.around(x, 2) for x in list(np.arange(0.1, 0.25, 0.05)) + [0.25]]
num_mutation_list = [np.around(x, 2) for x in list(np.arange(0.1, 0.3, 0.05)) + [0.3]]
mutation_rate_list = [round(x, 2) for x in list(np.arange(0.1, 0.4, 0.05))]
crossover_rate_list = [np.around(x, 2) for x in list(np.arange(0.6, 0.95, 0.05)) + [0.95]]

space  = [Categorical([10, 30, 50], name='num_individuals'),
          Categorical(mutation_rate_list, name='mutation_rate'),
          Categorical(crossover_rate_list, name='crossover_rate'),
          Categorical(num_elite_list, name='num_elite'),
          Categorical(num_mutation_list, name='num_mutation')]

In [13]:
for fold in range(len(train_pos)):
    val_pos, val_neg, val_facts = get_train_test([train_pos[fold]], [train_neg[fold]], [train_facts[fold]],  n_folds=3)
    
    @use_named_args(space)
    def objective(**params):
        res = brkga_var(new_src_struct, target, source, train_pos, train_neg, 
                      train_facts, kb_source, kb_target, pred_target,
                      NUM_GEN=14, pop_size=params['num_individuals'], crossover=params['crossover_rate'],
                      mutation=params['mutation_rate'], num_top=params['num_elite'], 
                      num_bottom=params['num_mutation'], revision='guided')

        return res[1][-1]
    res_gp = gp_minimize(objective, space, n_calls=10, random_state=0)

    print("Best score=%.4f " % res_gp.fun)
    print(f"FOLD {fold} - BEST RESULT: {res_gp.x}")
    
    test = []
    ttrain = []
    test_pos = []
    test_neg = [] 
    test_facts = []
    for index in range(0, len(train_pos)):
        if index == fold:
            ttrain = [train_pos[index], train_neg[index], train_facts[index]]
        else:
            test_pos.extend(train_pos[index])
            test_neg.extend(train_neg[index])
            test_facts.extend(train_facts[index])
    test = [test_pos, test_neg, test_facts]
    
    train_pos_gen = [ttrain[0], test[0]]
    train_neg_gen = [ttrain[1], test[1]]
    train_facts_gen = test[2]
    
    for _round in range(0, n_rounds):
        print(f"ROUND {str(_round+1)}")


        res_brkga_var = brkga_var(new_src_struct, target, source, 
                    train_pos, train_neg, train_facts, 
                    kb_source, kb_target, pred_target,
                    NUM_GEN=14, pop_size=res_gp.x[0], 
                    mutation=res_gp.x[1], crossover=res_gp.x[2],
                    num_top=res_gp.x[3], 
                    num_bottom=res_gp.x[4], revision='guided')

        final_results = {}
        final_results[f'{source}->{target}'] = res_brkga_var

        individuals = get_k_best_individuals(res_brkga_var[0].population, 3)
        
        n_ind = 1
        for individual in individuals:
            rrefine = []
            rtransfer = []
            print("INDIVIDUO ", n_ind)
            refine, transfer = get_refine_transfer(individual)
            rrefine.append(refine)
            rtransfer.append(transfer)
            res = []
            inf = []
            
            res_ =  test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
            res.append(res_)

            thisFile = f'boostsrl/test/results_{target}.db'
            base = os.path.splitext(thisFile)[0]
            os.rename(thisFile, base + ".txt")
            tt = open(f'boostsrl/test/results_{target}.txt', 'r').readlines()
            final = []
            for i in tt:
                final.append(i.replace('\n', ''))
            inf.append(final)


            final_results[f'test:{source}->{target}'] = res
            final_results[f'refine:{source}->{target}'] = rrefine
            final_results[f'transfer:{source}->{target}'] = rtransfer
            final_results[f'inf:{source}->{target}'] = inf
            save_groot_results(f'groot_experiments/{EXPERIMENT_NAME}_{source}_{target}/brkga_var/train_fold_{fold}/{str(res_gp.x[2])}_{str(res_gp.x[1])}_{str(res_gp.x[0])}_15_{str(res_gp.x[3])}_{str(res_gp.x[4])}', n_ind, final_results, source, target)
            n_ind += 1

/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


GENERATION:  0
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.304937, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  1
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.306507, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  2
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.302763, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  3
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.306627, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cl

MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.3086, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  8
MELHOR RESULTADO:  -0.205038
GENERATION:  0
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.302893, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  1
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.302082, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  2
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.308708, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0

BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.276392, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  7
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.276392, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  8
MELHOR RESULTADO:  -0.205038
GENERATION:  0
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.304957, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  1
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.304957, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cl

/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


...

MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.308795, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  6
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.306624, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  7
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.308924, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  8
MELHOR RESULTADO:  -0.205038
GENERATION:  0
MELHOR RESULTADO:  -0.095597
BEST: {'m_auc_pr': 0.095597, 'm_auc_roc': 0.867831, 'm_cll': -0.288996, 'm_rec': 1.0, 'm_pred': 0.2977985, 'm_f1': 0.174512, 's_auc_pr'

BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.273437, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  5
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.276413, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  6
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.276413, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  7
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.276693, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, '

/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


GENERATION:  0
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.306433, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  1
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.303742, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  2
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.30869, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  3
MELHOR RESULTADO:  -0.205038
BEST: {'m_auc_pr': 0.205038, 'm_auc_roc': 0.904593, 'm_cll': -0.305215, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll

MELHOR RESULTADO:  -0.205418
BEST: {'m_auc_pr': 0.205418, 'm_auc_roc': 0.905162, 'm_cll': -0.308555, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  7
MELHOR RESULTADO:  -0.205418
BEST: {'m_auc_pr': 0.205418, 'm_auc_roc': 0.905162, 'm_cll': -0.308555, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  8
MELHOR RESULTADO:  -0.205418
BEST: {'m_auc_pr': 0.205418, 'm_auc_roc': 0.905162, 'm_cll': -0.308555, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec': 0.0, 's_prec': 0.203374, 's_f1': 0.0}
GENERATION:  9
MELHOR RESULTADO:  -0.205418
BEST: {'m_auc_pr': 0.205418, 'm_auc_roc': 0.905162, 'm_cll': -0.308555, 'm_rec': 1.0, 'm_pred': 0.296626, 'm_f1': 0.170595, 's_auc_pr': 0.0, 's_auc_roc': 0.0, 's_cll': 0.0, 's_rec